## Import Dependencies

In [86]:
import pandas as pd
from IPython.display import display
%matplotlib inline
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [74]:
train_dataset = pd.read_csv('E:/学习/UCL/year-3/comp0036/final_train_data.csv')
test_dataset = pd.read_csv('E:/学习/UCL/year-3/comp0036/final_test_data.csv')

X_train = train_dataset.drop('FTR', axis=1)
y_train = train_dataset['FTR']
X_test = test_dataset.drop('FTR', axis=1)
y_test = test_dataset['FTR']

In [75]:
#for measuring training time
from time import time 
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, average='micro'), sum(target == y_pred) / float(len(y_pred))

def train_predict(clf, X_train, y_train, X_test, y_test):

    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

## Make Prediction

In [77]:
# Initialize and train the NB model
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(clf_nb, X_train, y_train, cv=5, scoring='accuracy')

# Print the accuracy scores for each fold
print("Cross-validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

# Evaluate the model on the test set
train_predict(clf_nb, X_train, y_train, X_test, y_test)

Cross-validation Scores: [0.51649485 0.50128932 0.54357916 0.51933987 0.54151625]
Mean Accuracy: 0.5244438891340526
Training a GaussianNB using a training set size of 9696. . .
Trained model in 0.0355 seconds
Made predictions in 0.0406 seconds.
0.5410478547854786 0.5410478547854786
F1 score and accuracy score for training set: 0.5410 , 0.5410.
Made predictions in 0.0112 seconds.
F1 score and accuracy score for test set: 0.5492 , 0.5492.


In [92]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Define the parameter distribution to search
param_dist = {
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(1, 5),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.3, 0.5),
    'n_estimators': randint(100, 200),
}

# Initialize xgb model with regularization parameters
clf_xgb = XGBClassifier(seed=82, reg_alpha=0.1, reg_lambda=0.1, objective='multi:softmax', num_class=3, 
                        eval_metric='mlogloss')

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=clf_xgb, param_distributions=param_dist, n_iter=10, scoring='f1_micro', random_state=42, cv=5
)

# Perform random search on the resampled training set
random_search.fit(X_train, y_train)

# Print the best parameters and corresponding score
print("Best Parameters: ", random_search.best_params_)
print("Best Score: {:.4f}".format(random_search.best_score_))

# Use the best model directly for early stopping
best_model_xgb = random_search.best_estimator_

# Set up evaluation sets for early stopping
X_val, X_test, y_val, y_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
evals = [(X_val, y_val)]

# Train the XGBoost model with early stopping
best_model_xgb.fit(X_train, y_train, eval_set=evals, early_stopping_rounds=20)

# Evaluate the model with the best hyperparameters on the test set
train_predict(best_model_xgb, X_train, y_train, X_test, y_test) 


Best Parameters:  {'colsample_bytree': 0.45212112147976885, 'learning_rate': 0.062475643163223786, 'max_depth': 4, 'n_estimators': 188, 'subsample': 0.8582458280396085}
Best Score: 0.8970
[0]	validation_0-mlogloss:1.06792


c:\Users\86182\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-mlogloss:1.03200
[2]	validation_0-mlogloss:0.99914
[3]	validation_0-mlogloss:0.97144
[4]	validation_0-mlogloss:0.94531
[5]	validation_0-mlogloss:0.91833
[6]	validation_0-mlogloss:0.88988
[7]	validation_0-mlogloss:0.86595
[8]	validation_0-mlogloss:0.84080
[9]	validation_0-mlogloss:0.81825
[10]	validation_0-mlogloss:0.79966
[11]	validation_0-mlogloss:0.78110
[12]	validation_0-mlogloss:0.75932
[13]	validation_0-mlogloss:0.74518
[14]	validation_0-mlogloss:0.72816
[15]	validation_0-mlogloss:0.71157
[16]	validation_0-mlogloss:0.69383
[17]	validation_0-mlogloss:0.68000
[18]	validation_0-mlogloss:0.66811
[19]	validation_0-mlogloss:0.65383
[20]	validation_0-mlogloss:0.64255
[21]	validation_0-mlogloss:0.62675
[22]	validation_0-mlogloss:0.61653
[23]	validation_0-mlogloss:0.61105
[24]	validation_0-mlogloss:0.60239
[25]	validation_0-mlogloss:0.59222
[26]	validation_0-mlogloss:0.58275
[27]	validation_0-mlogloss:0.56972
[28]	validation_0-mlogloss:0.56127
[29]	validation_0-mlogloss:0.